<h1>Coursera Data Science Capstone Project Part - 1<h1>
<h4>This notebook will be used to implement the Capstone projects titled <b><i>"The battle of neighbourhoods"</i></b></h4>

<h3>Segmenting and Clustering Neighborhoods in Toronto</h3>

In [1]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

print('Libraries imported')

Libraries imported


In [0]:
#Toronto Wikipedia link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#getting content from the html page
page_content = lh.fromstring(requests.get(url).content)
#accessing the table data
table_data = page_content.xpath("//tr")

In [17]:
header=[]
col_ind=0

#Creating header with empty list to add list of values under each column
for row in table_data[0]:
    col_ind+=1
    name=row.text_content()
    print (col_ind,name)
    header.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood

